In [52]:
import numpy as np
import cv2
import csv

In [53]:
def motion_detector(video_file=0, compress_to_size=(100,100), show_video=False, frame_step=2, max_frames=np.inf, output_file=""):
  
  frame_count = 0
  previous_frame = None
  cap = cv2.VideoCapture(video_file)
  preprocessed_file = []
  
  while True:
    frame_count += 1

    # 1. Load image; convert to RGB
    ret, frame = cap.read()

    # If video check when end or ESC is hit
    if frame is None or cv2.waitKey(30) == 27 or frame_count > max_frames:
      print(frame_count - 1)
      cap.release()
      
      if show_video:
        cv2.destroyAllWindows()

      # if a output file is given save the file
      if(output_file != ""):
        f = open(output_file, 'w')
        for item in preprocessed_file:
            f.write(','.join([str(x) for x in item]) + '\n')
        f.close()

      break

    frame = np.flip(frame, axis=1)
    frame = cv2.resize(frame, compress_to_size)
    frame = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2RGB)

    
    if ((frame_count % frame_step) == 0):

      # 2. Prepare image; grayscale and blur
      prepared_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5,5), sigmaX=0)
      
      # 3. Set previous frame and continue if there is None
      if (previous_frame is None):
        # First frame; there is no previous one yet
        previous_frame = prepared_frame
        continue
      
      # calculate difference and update previous frame
      diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
      previous_frame = prepared_frame

      # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
      # kernel = np.ones((5, 5))
      # diff_frame = cv2.dilate(diff_frame, kernel, 1)

      # 5. Only take different areas that are different enough (>20 / 255)
      thresh_frame = cv2.threshold(src=diff_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]

      # Save every frame
      preprocessed_file.append((thresh_frame.flatten()/255).astype(int))

      if(show_video):
        both_frame = np.concatenate((prepared_frame, thresh_frame), axis=1)
        cv2.imshow('webcam', both_frame)

In [54]:
# motion_detector(video_file='videos/right_hand_A2_V1.mp4', show_video=True, compress_to_size=(50,50), max_frames=90, output_file="test.csv")

52
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
